# ***Saving this notebook for later - use after setting up full workflow.***

# Pre-Pre-Processing

Before I start performing any EDA or modeling, I will pre-emptively split my data to have a separate training set of data.

Since the data consists of temporal data (hotel reservations across multiple years), I will split the data into two groups based on two datetime features: Booking Date and the Arrival date.

---

# Consulting ChatGPT

I asked ChatGPT to help me decide if one feature would be better than the other. It's responses are as follows:

## Booking Date

### Pros:
> * *Forecasting Lead Time:* Splitting by booking date allows you to model the lead time between booking and arrival, which is valuable for forecasting future demand well in advance.
> * *Real-world Scenario Modeling:* This approach mirrors real-world scenarios more closely, where predictions are made as bookings occur, regardless of the future arrival date.
> * *Dynamic Pricing and Revenue Management:* Particularly useful for applications in dynamic pricing and revenue management, where understanding booking patterns ahead of time can inform strategic decisions.

### Cons:
> * *Increased Complexity:* The time gap between booking and arrival dates introduces additional complexity, as you need to account for changes over time that could affect the stay (e.g., cancellations, modifications).



## Arrival Date

### Pros:
> * *Cleaner Data Relationships:* Utilizing the arrival date might result in cleaner, more straightforward modeling, as it directly ties the prediction to the period of service or product consumption.
> * *Seasonality and Trend Analysis:* Easier to model seasonal trends and effects, as the arrival date directly corresponds to the period being analyzed.
> * *Accuracy in Performance Metrics:* Predictions based on arrival date can be more closely aligned with actual occupancy and revenue, potentially improving model accuracy in terms of performance metrics.

### Cons:

> * *Reduced Forecasting Horizon:* The model may be less effective at predicting bookings well in advance since it's oriented around the arrival date. This could limit its usefulness for long-term planning.
> * *Possible Lag in Actionable Insights:* May not provide as much lead time for implementing strategies based on the predictions, such as staffing or promotional offers, since the focus is on the period closer to the actual stay.

---

# Consultation Conclusions

After considering ChatGPT's suggestions and insights, I will take a more greedy approach and create separate datasets for both the booking and arrival dates. This will give me more flexibility when modeling as I will have different time perspectives to utilize for different purposes (e.g., future forecasting vs. analyzing actualized performance).

---

# Date Preparation

Before I can split the datasets, I need to perform some slight feature engineering. The source datasets do not have an exact datetime feature for the arrival date, only for the booking date. I will use the separate Year, Month, and Day of Month features to create an `Arrival_Date` feature, then use this feature for splitting my data.

In [2]:
# raise Exception('REVISE THIS WORKBOOK - CHANGED TO FEATURE ENGINEERING; CONVERT BACK TO SUBSETTING DATA.')

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
## Enabling access to custom functions in separate directory

# Import necessary modules
import os
import sys

# Construct the absolute path to the 'src' directory
src_path = os.path.abspath(os.path.join('..', 'src'))

# Append the path to 'sys.path'
if src_path not in sys.path:
    sys.path.append(src_path)

import db_utils, eda

In [4]:
import pandas as pd

# Read Data - Engineered Datetime Data

In [5]:
df_data = pd.read_parquet('../data/engineered_data_dates.parquet', columns = ['UUID', 'ArrivalDate', 'BookingDate'])
df_data

,UUID,ArrivalDate,BookingDate
0,8ca998d6-fae7-4ee4-a706-3765721aaff5,2015-07-01,2014-07-24
1,e535835e-b19a-4e32-9e9f-6d70a0182d4b,2015-07-01,2013-06-24
2,9429383d-0efd-4c37-bb9b-0aaa63d5aade,2015-07-01,2015-06-24
3,dd6424ee-6838-4007-ad85-de9ff96be14b,2015-07-01,2015-06-18
4,50ff56ee-6a72-40dc-8ff1-4246b831c779,2015-07-01,2015-06-17
...,...,...,...
119385,2ccdf728-5829-47d6-ae2c-016f9706c24d,2017-08-30,2017-08-07
119386,be937240-f461-4eee-9971-a83c8c09bf04,2017-08-31,2017-05-21
119387,04e0baed-c9a6-487c-a1af-82cb0f830fe4,2017-08-31,2017-07-28
119388,4d96b250-c5c4-46e5-bf52-e4334991d81b,2017-08-31,2017-05-14


# Read Data from DuckDB

---


***DEPRECATED:* USE DATA FROM FEATURE ENGINEERING NOTEBOOK**

In [6]:
# # Path to the DuckDB database file
# db_path = '../data/hotel_reservations.duckdb'

# ## Select subset of data for review
# q = 'SELECT * FROM res_data LIMIT 10'

# with db_utils.duckdb_connection(db_path) as conn:
#     display(conn.execute(q).df())

In [7]:
# ## Convert Arrival columns to strings

# q = ('''
# SELECT uuid, ArrivalDateYear, ArrivalDateMonth, ArrivalDateDayOfMonth,
# StaysInWeekNights, StaysInWeekendNights, LeadTime 
# FROM res_data''')

# with db_utils.duckdb_connection(db_path) as conn:
#     df_data = conn.execute(q).df()

# # df_data = arrival_cols.astype(str)
# df_data.head()

# Feature Engineering: Arrival, Departure, and Booking Dates

## Arrival Date

In [8]:
# ## Create new column of strings formatted as YYYY-MM-DD, then convert to datetime

# arrival_details = ['ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth']

# df_data[arrival_details] = df_data[arrival_details].astype(str)

# df_data['ArrivalDate'] = df_data['ArrivalDateYear'].str.cat(df_data[['ArrivalDateMonth',
#                                                                      'ArrivalDateDayOfMonth']],
#                                                             '-')

# df_data['ArrivalDate'] = pd.to_datetime(df_data['ArrivalDate'], yearfirst = True)

# df_data.head()

## Departure Date

In [9]:
# timedelta_wknd = pd.to_timedelta(df_data.loc[:, 'StaysInWeekendNights'], unit = 'D')
# timedelta_wk = pd.to_timedelta(df_data.loc[:, 'StaysInWeekNights'], unit = 'D')

# df_data['DepartureDate'] = df_data.loc[:, 'ArrivalDate'] + timedelta_wk + timedelta_wknd

# df_data.head()

## Booking Date

In [10]:
# df_data['LeadTimeDelta'] = pd.to_timedelta(df_data['LeadTime'], unit = 'D')
# df_data['LeadTimeDelta']

In [11]:
# df_data['BookingDate'] = df_data['ArrivalDate'] - df_data['LeadTimeDelta']

# df_data.head(10)

In [12]:
# drop_cols = ['ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth',
# 'StaysInWeekNights', 'StaysInWeekendNights', 'LeadTime', 'LeadTimeDelta']
# df_data = df_data.drop(columns = drop_cols)
# df_data.head()

In [13]:
# df_data.to_parquet('../data/reservation_dates.parquet', compression = 'snappy')

# Subset with 10% Threshold for Arrival_Date and Booking_Date

Despite the sizable number of rows for each hotel's dataset, the Arrival_Date feature is limited in scope - covering a little over 2 years' worth of data. This limited time range restricts the possible forecasting windows, particularly as I intend to reserve a certain number of days for the validation sets.

My compromise between availability and integrity is to subset each hotel's separate datasets by 10% of the total dataset. This will preserve the size of the original data's timeframe while providing a reasonable time window for future forecasting.

In [14]:
def split_and_save_dataset_by_percentage(df,
                                         date_column,
                                         percentage=10,
                                         save_path='./data'):
    """
    Split the dataset into training and holdout sets based on a specified percentage of the total rows
    and save them. The holdout set will contain the last 'percentage' of rows based on the datetime column.
    """
    
    # Work on a copy of the DataFrame to preserve the original data
    df_copy = df.copy()
    
    # Ensure the date column is in datetime format and set it as the index
    df_copy[date_column] = pd.to_datetime(df_copy[date_column])
    df_copy = df_copy.sort_values(by=[date_column], ascending = True)

    # Calculate the index to split on last 'percentage' of rows
    split_idx = int(len(df_copy) * (1 - (percentage / 100)))

    # Split the dataset and reset index for future use
    train_df = df_copy.iloc[:split_idx,:]
    train_df = train_df.reset_index(drop=True)
    
    holdout_df = df_copy.iloc[split_idx:,:]
    holdout_df = holdout_df.reset_index(drop=True)


    # No need to reset index if you want to keep the datetime column in its place
    # Define file paths
    training_path = f'{save_path}/{date_column}_Training.parquet'
    holdout_path = f'{save_path}/{date_column}_Validation.parquet'

    # Save the datasets
    train_df.to_parquet(training_path, engine='pyarrow', compression='snappy')
    print(f'Saved training data for {date_column}')
    holdout_df.to_parquet(holdout_path, engine='pyarrow', compression='snappy')
    print(f'Saved validation data for {date_column}.\n')


In [15]:
date_columns = ['ArrivalDate', 'BookingDate']

for date_column in date_columns:
    split_and_save_dataset_by_percentage(df_data,
                                         date_column,
                                         percentage = 10,
                                         save_path = f'../data/')

Saved training data for ArrivalDate
Saved validation data for ArrivalDate.

Saved training data for BookingDate
Saved validation data for BookingDate.



# Results

After running this code for both of the hotel datasets, I now have separate training and validation datasets for each of the hotels' Arrival_Date and Booking_Date features. The resulting datasets will be effective for modeling and forecasting in later notebooks.

In [16]:
# ## Create timedelta series based on number of weekday/end nights.
# timedelta_wknd = pd.to_timedelta(df_data.loc[:, 'StaysInWeekendNights'], unit = 'D')
# timedelta_wk = pd.to_timedelta(df_data.loc[:, 'StaysInWeekNights'], unit = 'D')

In [17]:
# table_name = 'res_data'

# with db_utils.duckdb_connection(db_path) as conn:
#     column_info = conn.execute(f"PRAGMA table_info({table_name})").fetchall()

#     # Print information about each column
#     for column in column_info:
#         column_name, column_dtype = column[1], column[2]
#         print(f"Column Name: {column_name}, Data Type: {column_dtype}")

# _ = db_utils.get_col_dtypes('res_data', db_path)

In [18]:
# create_dates_table = '''CREATE TABLE reservation_dates (
#     UUID VARCHAR,
#     datetime_column TIMESTAMP,
#     PRIMARY KEY (UUID),
#     FOREIGN KEY (UUID) REFERENCES res_data (UUID)
# );
# '''
# create_dates_table

In [19]:
# db_utils.alter_table(create_dates_table, db_path)

In [20]:
# db_utils.alter_table('ALTER TABLE res_data ADD PRIMARY KEY (UUID);', db_path)

In [21]:
# alter_commands = "ALTER TABLE res_data ADD COLUMN Arrival_Date DATE"
# with duckdb_connection(db_path) as conn:
#     conn.execute(alter_commands)
#     print('Completed successfully.')

# drop_col = 'ALTER TABLE res_data DROP COLUMN Arrival_Date;'
# db_utils.alter_table(drop_col, db_path)

# alt_tbl_cmd = "ALTER TABLE res_data ADD COLUMN Arrival_Date TIMESTAMP"
# db_utils.alter_table(alt_tbl_cmd, db_path)

In [22]:
# q = 'SELECT Arrival_Date FROM res_data'
# with db_utils.duckdb_connection(db_path) as conn:
#     display(conn.execute(q).df())

In [23]:
# q = 'SELECT Arrival_Date FROM res_data'
# with db_utils.duckdb_connection(db_path) as conn:
#     result = conn.execute(q).fetchall()

#     for row in result:
#         print(row)

In [24]:
# ## Concatenate new column
# df_data = pd.concat([df_data, arrival_date_dt], axis = 1)
# df_data.head()

In [25]:
# try:
#     df_data = df_data.drop(columns=[ 'ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth'])
# except:
#     pass

# df_data.head()

In [26]:
# q = 'SELECT UUID, StaysInWeekNights, StaysInWeekendNights FROM res_data'

# with db_utils.duckdb_connection(db_path) as conn:
#     stays_cols = conn.execute(q).df()
    
# stays_cols.head()

In [27]:
# # pd.concat([arrival_date_cols_str, stays_cols], axis = 1)

# df_data = arrival_date_cols_str.merge(stays_cols, how = 'left', on = 'UUID')
# df_data.head()

In [28]:


## Calculate the departure date by adding the timedeltas to the arrival date
# departure_date = df_data.loc[:, 'ArrivalDate'] + timedelta_wk + timedelta_wknd
# departure_date.name = 'DepartureDate'
# departure_date.head()

# df_data['DepartureDate'] = df_data.loc[:, 'ArrivalDate'] + timedelta_wk + timedelta_wknd
# df_data.head()

In [29]:
# try:
#     df_data = df_data.drop(columns=['StaysInWeekendNights', 'StaysInWeekNights'])
# except:
#     pass

# df_data.head()

In [30]:
# q = 'SELECT LeadTime from res_data'

# with db_utils.duckdb_connection(db_path) as conn:
#     leadtime = conn.execute(q).df()
#     print('Completed successfully.')

In [31]:
# df_data = pd.concat([df_data, leadtime['LeadTime']], axis = 1)

# df_data['LeadTimeDelta'] = pd.to_timedelta(df_data['LeadTime'], unit = 'D')
# df_data.head()

In [32]:
# df_data['BookingDate'] = df_data['ArrivalDate'] - df_data
# df_data.head(10)

In [33]:
# df_data = df_data.drop(columns = ['LeadTime']) ## Maintain for future modeling

# df_data.head(10)

In [34]:
# df_data['Arrival_Date'].max() - df_data['Arrival_Date'].min()

In [35]:
# df_data['Booking_Date'].max() - df_data['Booking_Date'].min()